# Mecab ユーザー辞書作り

##### 手順

1. ユーザ辞書の準備として、CSVファイルを作成
2. コマンドラインで DIC ファイルを作成
3. 作成した辞書にパスを通す

### 1. ユーザ辞書作成の準備として、CSVファイルを作成

In [4]:
require 'csv'

false

In [5]:
original_data = {
  elementary_schools: 'e_schools.csv',
  abolished_e_schools: 'abolished_e_schools.csv',
  jh_schools: 'jh_schools.csv',
  abolished_jh_schools: 'abolished_jh_schools.csv',
  high_schools: 'high_schools.csv',
}

{:elementary_schools=>"e_schools.csv", :abolished_e_schools=>"abolished_e_schools.csv", :jh_schools=>"jh_schools.csv", :abolished_jh_schools=>"abolished_jh_schools.csv", :high_schools=>"high_schools.csv"}

In [6]:
CSV.open("schools.csv", 'w') do |csv|
    original_data.each do |type, filename|
        next unless File.file? filename
        open(filename).each do |title|
            title.strip!
            
            next if title =~ %r(^[+-.$()?*/&%!"'_,]+)
            next if title =~ /^[-.0-9]+$/
            next if title =~ /曖昧さ回避/
            next if title =~ /_\(/
            next if title =~ /^PJ:/
            next if title =~ /の登場人物/
            next if title =~ /一覧/
                                                     
            title_length = title.length
                                                     
            if title_length > 3
             score = [-36000.0, -400 * (title_length ** 1.5)].max.to_i
             csv << [title, nil, nil, score, '名詞', '一般', '*', '*', '*', '*', title, '*', '*', type]
            end
        end
     end
end


{:elementary_schools=>"e_schools.csv", :abolished_e_schools=>"abolished_e_schools.csv", :jh_schools=>"jh_schools.csv", :abolished_jh_schools=>"abolished_jh_schools.csv", :high_schools=>"high_schools.csv"}

参考:http://qiita.com/ynakayama/items/388c82cbe14c65827769

### 2. コマンドラインで DIC ファイルを作成

コマンドラインで以下を実行する。

・/usr/local/Cellar/mecab/0.996/libexec/mecab/mecab-dict-index -d /usr/local/lib/mecab/dic/ipadic -u /Users/runa/python/All_something_in_Japan/schools_new.dic -f utf-8 -t utf-8 /Users/runa/python/All_something_in_Japan/schools.csv

mecabの辞書を置く場所にコピー

・cp schools_new.dic /usr/local/lib/mecab/dic

### 3. 作成した辞書にパスを通す

・vim /usr/local/etc/mecabrc

・cat /usr/local/etc/mecabrc

;
; Configuration file of MeCab
;
; 
$Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;

dicdir =  /usr/local/lib/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

userdic_wikipedia = /usr/local/lib/mecab/dic/custom.dic

userdic_schools = /usr/local/lib/mecab/dic/schools.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n

### 確認

In [7]:
require 'uri'
require 'sqlite3'
require 'natto'
require 'mecab'

true

In [8]:
userdic_path = "./schools_new.dic"

"./schools_new.dic"

In [9]:
c = MeCab::Tagger.new("-u #{userdic_path}")

#<MeCab::Tagger:0x007fb76a308490 @__swigtype__="_p_MeCab__Tagger">

In [10]:
print c.parse("白山台中")

白山台中	名詞,一般,*,*,*,*,白山台中,*,*,jh_schools
EOS
